In [0]:
dbutils.widgets.text("p_data_date", "")
v_data_date = dbutils.widgets.get("p_data_date")

In [0]:
dbutils.secrets.list(scope="Fomular1-scope")

In [0]:
fomular1_key = dbutils.secrets.get(scope="Fomular1-scope", key="formuladl1-key-account")

In [0]:
spark.conf.set("fs.azure.account.key.fomular1dl1.dfs.core.windows.net", fomular1_key)

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, FloatType
from pyspark.sql.functions import col, lit, to_timestamp, concat, current_timestamp, to_date

In [0]:
results_schema = StructType(fields=[StructField("resultId", IntegerType(), False),
                                    StructField("raceId", IntegerType(), True),
                                    StructField("driverId", IntegerType(), True),
                                    StructField("constructorId", IntegerType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("grid", IntegerType(), True),
                                    StructField("position", IntegerType(), True),
                                    StructField("positionText", StringType(), True),
                                    StructField("positionOrder", IntegerType(), True),
                                    StructField("points", FloatType(), True),
                                    StructField("laps", IntegerType(), True),
                                    StructField("time", StringType(), True),
                                    StructField("milliseconds", IntegerType(), True),
                                    StructField("fastestLap", IntegerType(), True),
                                    StructField("rank", IntegerType(), True),
                                    StructField("fastestLapTime", StringType(), True),
                                    StructField("fastestLapSpeed", FloatType(), True),
                                    StructField("statusId", StringType(), True)])



In [0]:
result_df = spark.read.schema(results_schema).json(f"abfss://raw@fomular1dl1.dfs.core.windows.net/{v_data_date}/results.json")

In [0]:
final_df = result_df.withColumnRenamed("resultId", "result_id") \
                   .withColumnRenamed("raceId", "race_id") \
                   .withColumnRenamed("driverId", "driver_id") \
                   .withColumnRenamed("constructorId", "constructor_id") \
                   .withColumnRenamed("positionText", "position_text") \
                   .withColumnRenamed("positionOrder", "position_order")\
                    .withColumnRenamed("fastestLap", "fastest_lap") \
                   .withColumnRenamed("fastestLapTime", "fastest_lap_time") \
                   .withColumnRenamed("fastestLapSpeed", "fastest_lap_speed") \
                   .withColumn("ingestion_date", current_timestamp())\
                    .drop("statusId")


final_df.show()
                   

In [0]:
final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.results")